In [1]:
import qutip
import numpy as np
import matplotlib.pyplot as plt
#from scipy.integrate import quad
from einops import rearrange, reduce, repeat
from scipy.linalg import expm 


A code to write out the MPO form of a beamsplitter 
<br> Note; the MPOS are labelled clockwise, with the 'top' leg labelled first, in comments - where no leg

In [82]:
np.set_printoptions(precision=3)
#, suppress=True)

In [83]:
input = '11110000'

In [84]:
#dim of fock space of each rail
n_dim = 3
#no of rails , each modelled as a cavity
n_rails = len(input)

In [85]:
#needed when we actually are modelling capturing an atoms output
'''
atom_a = qutip.destroy(n_dim).full()
atom_adag = qutip.destroy(n_dim).dag().full()
'''

#Creation and anihilation op for cavities
#For beamsplitter all 'rails' taking same size, so one set of a/adag needed
a = qutip.destroy(n_dim).full()
adag = qutip.destroy(n_dim).dag().full()

# Functions

In [86]:
def svd_red(mpo_list, no_cav = n_rails, cav_dim = n_dim):
    #Assuming there are at least two cavitites

    red_mpo_list = list()
    lim = 20

    if no_cav > 2:
        u, s, v = np.linalg.svd(rearrange(np.einsum('a b c, d e f b -> a d e c f', mpo_list[0], mpo_list[1]), 'a d e c f-> (a c) (d e f)'), full_matrices=False)
        bond_dim = min(lim, len(s))
        u, s, v = u[:,:bond_dim], s[:bond_dim], v[:bond_dim,:]
        red_mpo_list.append(rearrange(u,'(a c) g-> a g c', a=cav_dim, c=cav_dim))
        red_mpo_list.append(rearrange(np.diag(s)@v, 'g (d e f)-> d e f g', d=cav_dim, f=cav_dim))
        
        for i in range(1,no_cav-1):

            if i == no_cav-2:
                u, s, v = np.linalg.svd(rearrange(np.einsum('b c d a, e f c -> b e d f a', red_mpo_list[-1], mpo_list[i+1]), 'b e d f a-> (a b d) (e f)'), full_matrices=False)
                bond_dim = min(lim, len(s))
                u, s, v = u[:,:bond_dim], s[:bond_dim], v[:bond_dim,:]
                red_mpo_list.pop()
                red_mpo_list.append( rearrange(u, '(a b d) h-> b h d a', b=cav_dim, d=cav_dim))
                red_mpo_list.append( rearrange(np.diag(s)@v, 'h (e f)-> e f h', e=cav_dim, f=cav_dim))

            else:
                u, s, v = np.linalg.svd(rearrange(np.einsum('b c d a, e f g c-> b e f d g a', red_mpo_list[-1], mpo_list[i+1]), 'b e f d g a-> (a b d) (e f g)'), full_matrices=False)
                bond_dim = min(lim, len(s))
                print(s)
                u, s, v = u[:,:bond_dim], s[:bond_dim], v[:bond_dim,:]
                red_mpo_list.pop()
                red_mpo_list.append( rearrange(u, '(a b d) h-> b h d a', b=cav_dim, d=cav_dim))
                red_mpo_list.append( rearrange(np.diag(s)@v, 'h (e f g)-> e f g h', e=cav_dim, g=cav_dim))
                #print(s)
    else :
        u, s, v = np.linalg.svd(rearrange(np.einsum('a b c, d e b -> a d e c', mpo_list[0], mpo_list[1]), 'a d e c-> (a c) (d e)'), full_matrices=False)
        bond_dim = min(lim, len(s))
        u, s, v = u[:,:bond_dim], s[:bond_dim], v[:bond_dim,:]
        red_mpo_list.append(rearrange(u,'(a c) g-> a g c', a=cav_dim))
        red_mpo_list.append(rearrange(np.diag(s)@v, 'g (d e)-> d e g', d=cav_dim))


    return red_mpo_list

In [87]:
def exp_value(mpo_list, rail = -1):

    mpo_copy = list(mpo_list)

    if rail == 0:
        mpo_copy[0] = np.einsum( 'a b, b c d-> a c d', adag@a, mpo_copy[0])
    elif rail == n_rails-1:
        mpo_copy[rail] = np.einsum( 'a c, c d b-> a d b', adag@a, mpo_copy[rail])
    elif rail < 0:
        pass
    else:
        mpo_copy[rail] = np.einsum( 'a c, c d e b-> a d e b', adag@a, mpo_copy[rail])

    mpo_copy[0] = np.einsum( 'a b a', mpo_copy[0])
    for i in range(1,n_rails):
        if i == (n_rails-1):
            mpo_copy[i] =  np.einsum('a, b b a', mpo_copy[i-1], mpo_copy[i])
        else:
            mpo_copy[i] = np.einsum('a, b c b a', mpo_copy[i-1], mpo_copy[i])

    return mpo_copy[-1]

In [88]:
#Constructing the MPO of the beamsplitter

H_s1 = rearrange(np.array([a, adag]), 'c a b-> a c b')
H_s2 = rearrange(np.array([adag, a]), 'c a b-> a b c')
H_mpo =  rearrange( np.einsum('a b c, x y b -> a x c y', H_s1, H_s2), 'a x c y -> (a x) (c y)' )
beam_matrix =  expm(-0.25j*np.pi*H_mpo)
swap_matrix = expm(-0.5j*np.pi*H_mpo)

u, s, v = np.linalg.svd( rearrange(beam_matrix, '(a x) (c y) -> (a c) (x y)', a=n_dim, c=n_dim ) , full_matrices=False)
beam_mpo1 = rearrange(u, '(a d) x -> a x d', a=n_dim)
beam_mpo2 = rearrange(np.diag(s)@v, 'g (s t)-> s t g', s=n_dim)

bond_ord = beam_mpo2.shape[2]
beam_mpo_mid = np.array( [ np.array([0*np.eye(n_dim),]*bond_ord) ,]*bond_ord )
for i in range(bond_ord):
    beam_mpo_mid[i,i,:,:] = np.eye(n_dim)
beam_mpo_mid = rearrange( beam_mpo_mid, 'a b c d ->c b d a' )

In [89]:
u2, s2, v2 = np.linalg.svd( rearrange(swap_matrix, '(a x) (c y) -> (a c) (x y)', a=n_dim, c=n_dim ) , full_matrices=False) 
swap_mpo1 = rearrange(u, '(a d) x -> a x d', a=n_dim)
swap_mpo2 = rearrange(np.diag(s)@v, 'g (s t)-> s t g', s=n_dim)

In [90]:
def mpo_mul(mpo_list1, mpo_list2):
    out_mpo = list()
    no_mpo = len(mpo_list1)

    out_mpo.append( rearrange(np.einsum('a b c, c d e -> a b d e', mpo_list1[0], mpo_list2[0]), 'a b d e-> a (b d) e') )
    for i in range(1, no_mpo):
        if i==no_mpo-1:
            out_mpo.append( rearrange(np.einsum('a b c, b d e-> a d c e', mpo_list1[i], mpo_list2[i]), 'a d c e-> a d (c e)'))
        else:
            out_mpo.append( rearrange( np.einsum( ' a b c d, c e f g-> a b e f d g', mpo_list1[i], mpo_list2[i]), 'a b e f d g-> a (b e) f (d g)'))
    return svd_red(out_mpo)

In [91]:
def split2(mpo_list, apply_1, apply_2):

    beam_list = list()
#construxt beam list acc the 2 numbers
    for i in range(apply_1):
        beam_list.append(np.eye(n_dim)[:,np.newaxis,:,np.newaxis])
    beam_list += [beam_mpo1[:,:,:,np.newaxis],] + [beam_mpo_mid, ]*(apply_2-apply_1-1) + [beam_mpo2[:,np.newaxis,:,:],]
    for i in range(apply_2+1, n_rails):
        beam_list.append(np.eye(n_dim)[:,np.newaxis,:,np.newaxis])
    beam_list[0] = beam_list[0][:,:,:,0]
    beam_list[-1] = beam_list[-1][:,0,:,:]
    beam_list_conj = [i.conj() for i in beam_list]
    
    for i in range(n_rails):    
        print(beam_list[i].shape, exp_value(beam_list))
        print(mpo_list[i].shape)



    mpo_list1 = mpo_mul(beam_list, mpo_list)
    
    for i in range(n_rails):    
        print(mpo_list1[i].shape)

    mpo_list2 = mpo_mul(mpo_list1, beam_list_conj)
    return mpo_list2

In [92]:
def split(mpo_list, apply_1, apply_2):
    # assumption that apply_1 and _2 are in appropriate bounds, also _2 > _1
    #both are the indices of the MPO where the beamsplitter is actually applied

    #out_mpo = list(mpo_list)
    
    out_mpo = list()
    for i in range(n_rails):
        if i<apply_1 :
            out_mpo.append(np.copy(mpo_list[i]))
        elif i <= apply_2:
            out_mpo.append(np.empty)
        else: 
            out_mpo.append(np.copy(mpo_list[i]))
    
    
    if apply_1 == 0:
        out_mpo[apply_1] = rearrange (np.einsum('a b c, c d e, e f g -> a b d f g', beam_mpo1, mpo_list[apply_1], beam_mpo1.conj()), 'a b d f g -> a (b d f) g' )
    else : 
        out_mpo[apply_1] = rearrange (np.einsum('a b c, c d e f, e g h -> a b d g h f', beam_mpo1, mpo_list[apply_1], beam_mpo1.conj()), 'a b d g h f -> a (b d g) h f' )
    for i in range(apply_1+1, apply_2):
        out_mpo[i] = rearrange (np.einsum('a b c d, c e f g, f i j k -> a b e i j d g k', beam_mpo_mid, mpo_list[i], beam_mpo_mid.conj()), 'a b e i j d g k -> a (b e i) j (d g k)' )
    if apply_2 == (n_rails-1) :
        out_mpo[apply_2] = rearrange (np.einsum('a b c, b e f, e g h -> a g c f h', beam_mpo2, mpo_list[apply_2], beam_mpo2.conj()), 'a g c f h -> a g (c f h)' )
    else:
        out_mpo[apply_2] = rearrange (np.einsum('a b c, b d e f, e g h -> a d g c f h', beam_mpo2, mpo_list[apply_2], beam_mpo2.conj()), 'a d g c f h -> a d g (c f h)' )
    
    return out_mpo    

In [93]:
def swap(mpo_list, apply_1, apply_2):
    # assumption that apply_1 and _2 are in appropriate bounds, also _2 > _1
    #both are the indices of the MPO where the beamsplitter is actually applied

    #out_mpo = list(mpo_list)
    
    out_mpo = list()
    for i in range(n_rails):
        if i<apply_1 :
            out_mpo.append(np.copy(mpo_list[i]))
        elif i <= apply_2:
            out_mpo.append(np.empty)
        else: 
            out_mpo.append(np.copy(mpo_list[i]))
    
    
    if apply_1 == 0:
        out_mpo[apply_1] = rearrange (np.einsum('a b c, c d e, e f g -> a b d f g', swap_mpo1, mpo_list[apply_1], swap_mpo1.conj()), 'a b d f g -> a (b d f) g' )
    else : 
        out_mpo[apply_1] = rearrange (np.einsum('a b c, c d e f, e g h -> a b d g h f', swap_mpo1, mpo_list[apply_1], swap_mpo1.conj()), 'a b d g h f -> a (b d g) h f' )
    for i in range(apply_1+1, apply_2):
        out_mpo[i] = rearrange (np.einsum('a b c d, c e f g, f i j k -> a b e i j d g k', beam_mpo_mid, mpo_list[i], beam_mpo_mid.conj()), 'a b e i j d g k -> a (b e i) j (d g k)' )
    if apply_2 == (n_rails-1) :
        out_mpo[apply_2] = rearrange (np.einsum('a b c, b e f, e g h -> a g c f h', swap_mpo2, mpo_list[apply_2], swap_mpo2.conj()), 'a g c f h -> a g (c f h)' )
    else:
        out_mpo[apply_2] = rearrange (np.einsum('a b c, b d e f, e g h -> a d g c f h', swap_mpo2, mpo_list[apply_2], swap_mpo2.conj()), 'a d g c f h -> a d g (c f h)' )
    
    return out_mpo    

# Initialisation

n_dim here is the size of the fock space :

In [94]:

#Needed when atom for projection/MPO initialisation
'''
atom_1 = qutip.fock(n_dim,1).full()[:,0]
atom_0 = qutip.fock_dm(n_dim,1).full()
'''

#MPS to take projections, excited and ground
cavproj_1 = qutip.fock(n_dim,1).full()[:,0]
cavproj_0 = qutip.fock(n_dim,0).full()[:,0]
#cavproj_2 = qutip.fock(n_dim,2).full()[:,0]

#MPO starting point of each of the signal rails; excited state; one photon
cav_1 = qutip.fock_dm(n_dim,1).full()
#MPO starting point of each of the ancilla rails; ground state; zero photons
cav_0 = qutip.fock_dm(n_dim,0).full()
#cav_2 = qutip.fock_dm(n_dim,2).full()


In [95]:
#Generating MPO of rails

#atom-cavity list
rail_list = list()

rail_list.append(qutip.fock_dm(n_dim,1).full()[:,np.newaxis,:])
#d emp_leg d -
for i in range(n_rails-1):
    if i == n_rails-2:
        rail_list.append(qutip.fock_dm(n_dim,1).full()[ :, :, np.newaxis])
        # d - d emp_leg
    else:
        rail_list.append(qutip.fock_dm(n_dim,0).full()[ :, np.newaxis, :, np.newaxis])
        # d emp_leg d emp_leg



In [96]:
exp_value(rail_list)

(1+0j)

In [97]:
rail_list = svd_red(rail_list)

[1. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [98]:
exp_value(rail_list)

(1+0j)

In [99]:
bell_state = list()

for i in input:
    if i == '0':
        bell_state.append(qutip.fock_dm(n_dim,0).full()[ :, np.newaxis, :, np.newaxis])
    else:
        bell_state.append(qutip.fock_dm(n_dim,1).full()[ :, np.newaxis, :, np.newaxis])
    
bell_state[0] = bell_state[0][:,:,:,0]
bell_state[-1] = bell_state[-1][:,0,:,:]

bell_state = svd_red(bell_state)

[1. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [100]:
bell_state2 = list(bell_state)

In [101]:
exp_value(bell_state2)

(1+0j)

In [102]:
bell_state2 = split2(bell_state2, 0, 4)

(3, 9, 3) (3217.9616869699857+3.9140086768856853e-31j)
(3, 9, 3)
(3, 9, 3, 9) (3217.9616869699857+3.9140086768856853e-31j)
(3, 9, 3, 9)
(3, 9, 3, 9) (3217.9616869699857+3.9140086768856853e-31j)
(3, 9, 3, 9)
(3, 9, 3, 9) (3217.9616869699857+3.9140086768856853e-31j)
(3, 9, 3, 9)
(3, 1, 3, 9) (3217.9616869699857+3.9140086768856853e-31j)
(3, 9, 3, 9)
(3, 1, 3, 1) (3217.9616869699857+3.9140086768856853e-31j)
(3, 9, 3, 9)
(3, 1, 3, 1) (3217.9616869699857+3.9140086768856853e-31j)
(3, 9, 3, 9)
(3, 3, 1) (3217.9616869699857+3.9140086768856853e-31j)
(3, 3, 9)
[3.000e+00 3.000e+00 3.000e+00 1.947e-15 2.997e-16 2.997e-16 2.997e-16
 2.997e-16 2.997e-16 2.997e-16 2.997e-16 2.997e-16 2.997e-16 2.997e-16
 2.997e-16 2.997e-16 2.997e-16 2.997e-16 2.997e-16 2.997e-16 2.997e-16
 2.997e-16 2.997e-16 2.997e-16 2.997e-16 2.997e-16 2.997e-16 2.997e-16
 2.997e-16 2.997e-16 2.997e-16 2.997e-16 2.997e-16 2.997e-16 2.997e-16
 2.997e-16 2.997e-16 2.997e-16 2.997e-16 2.997e-16 2.997e-16 2.997e-16
 2.997e-16 2.997e-

In [105]:
exp_value(bell_state2)

(0.20598006844926442-0.012087189178572263j)

In [104]:
bell_state2 = split2(bell_state2, 2, 6)

(3, 1, 3) (3217.9616869699857+3.9140086768856836e-31j)
(3, 9, 3)
(3, 1, 3, 1) (3217.9616869699857+3.9140086768856836e-31j)
(3, 20, 3, 9)
(3, 9, 3, 1) (3217.9616869699857+3.9140086768856836e-31j)
(3, 20, 3, 20)
(3, 9, 3, 9) (3217.9616869699857+3.9140086768856836e-31j)
(3, 20, 3, 20)
(3, 9, 3, 9) (3217.9616869699857+3.9140086768856836e-31j)
(3, 20, 3, 20)
(3, 9, 3, 9) (3217.9616869699857+3.9140086768856836e-31j)
(3, 20, 3, 20)
(3, 1, 3, 9) (3217.9616869699857+3.9140086768856836e-31j)
(3, 9, 3, 20)
(3, 3, 1) (3217.9616869699857+3.9140086768856836e-31j)
(3, 3, 9)
[2.733e+00 2.733e+00 2.629e+00 2.214e+00 2.214e+00 2.093e+00 2.093e+00
 2.066e+00 1.903e+00 1.619e+00 1.607e+00 1.607e+00 9.737e-01 9.737e-01
 7.059e-01 6.321e-01 6.321e-01 5.591e-01 5.591e-01 2.861e-01 1.517e-15
 1.442e-15 1.140e-15 9.120e-16 7.671e-16 6.171e-16 4.964e-16 4.915e-16
 4.491e-16 3.650e-16 2.644e-16 2.644e-16 2.644e-16 2.644e-16 2.644e-16
 2.644e-16 2.644e-16 2.644e-16 2.644e-16 2.644e-16 2.644e-16 2.644e-16
 2.644e-

In [ ]:
bell_state2 = split2(bell_state2, 1, 5)

In [ ]:
bell_state2 = split2(bell_state2, 0, 4)

In [107]:
for i in range(n_rails):    
    print(bell_state2[i].shape)

(3, 9, 3)
(3, 20, 3, 9)
(3, 20, 3, 20)
(3, 20, 3, 20)
(3, 20, 3, 20)
(3, 20, 3, 20)
(3, 9, 3, 20)
(3, 3, 9)


In [ ]:
bell_state2 = svd_red(bell_state2)

In [ ]:
bell_state2 = split2(bell_state2, 4, 5)
bell_state2 = svd_red(bell_state2)
bell_state2 = split2(bell_state2, 6, 7)
bell_state2 = svd_red(bell_state2)
bell_state2 = split2(bell_state2, 4, 7)
bell_state2 = svd_red(bell_state2)
bell_state2 = split2(bell_state2, 5, 6)